<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/release/landlab_header.png"></a>

# Test comparing the original OverlandFlow Landlab component to our updated version of OverlandFlow
This notebook was created by Sam Anderson, Mikey Sison and Angel Monsalve.

<hr>
<small>For tutorials on learning Landlab, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a></small>
<hr>

**What is this notebook?**

This notebook generates results to compare the behaivor of the overland flow model presented by Adams et al. (2017) to the updated version used here to eventually couple with a sediment transport component. 

Our version of OverlandFlow was tested using the analytical solution for wave propagation on a flat surface case as first presented by Adams et al. (2017) and originally developed by Bates et al. (2010) and Hunter et al. (2005).
The idea of this test is to check if our version behaves in the same manner thand as the original OverlandFlow does under unsteady flow conditions. Our simulations were conducted in a flat domain of 6000 m long (x direction) 
and 800 m high (y direction) in which we used three different uniform grid configurations (∆x=∆y) with spacings of ∆x=5, 10, and 50 m. At the beginning of the simulation the domain was filled with a thin film of water of 1 mm. 
A uniform roughness coefficient was used (n=0.03). Boundary conditions at the top, right, and bottom edges of the domain were set to closed. Flow enters through the left edge (x=0 m) with a constant velocity of u=1 m/s and v=0 
m/s and water depth changes in time according to h(x,t)=h(0,t)=(7⁄3 n^2 u^3 t)^(3⁄7). 

After 3600 s of simulation we obtained identical results to those presented by Adams et al. (2017), indicating that our modifications to OverlandFlow do not affect the core of the flow solver (Figure 6). A detailed validation
is out of the scope our test and analysis but for more details on the validation the reader is referred to Adams et al. (2017).

More background on the model used here and the results presented will be published in the future and the citation will be added here:

**TITLE HERE**

The code used in this exercise is taken from the above reference.



**Now on to the code.**
* Below we import Landlab components, functions for importing data, numpy and plotting tools. You should not need to change this.

In [ ]:
## Code Block 1

%reset -f
import numpy as np
import pandas as pd
import copy
from matplotlib import pyplot as plt
from landlab.components import OverlandFlowSpatiallyVariableInputs, RiverBedDynamics
from landlab.io import read_esri_ascii
from landlab import imshow_grid


Now we import the data for the watershed we want to route flow on, as well as some model parameters 

In [ ]:
## Code Block 2

bedElevation =      'bedElevationDEM.asc'         # ASCII raster DEM containing the bed surface elevation
#bedElevation =      'bedElevationDEM.asc'         # ASCII raster DEM containing the bed surface elevation, uncomment for aggradation 

gsd = pd.read_excel('bedGSD.xlsx',sheet_name='GSD',skiprows=0).values

dtPrecision = 3         # Avoids rounding errors
max_dt = 5              # Overland flow will use the min time step between this value and the automatically calculated. Use seconds.
tPlot = 950400          # Plots will be obtained every this seconds **THIS VALUE WAS CHANGED FROM THE ORIGINAL**
storeData = 86400      # Stores results every this time
tmax = 121*86400          # Maximum simulation time

# Flow, bed, and upstream simulation conditions 
n = 0.03874                         # Manning's n 0.038577
upstreamSedSupply = -0.0012         # bedload rate at inlet

# Link Id in which sediment supply and discharge enters
link_Inlet = np.array([83])

# Node Id in Water depth is specified
Node_Inlet = np.array([52])

# Node ID for fixed Nodes
fixedNodesId = np.array((1,4))

In [ ]:
## Code Block 3

# Creates fields and instantiate the component
OverlandFlowSpatiallyVariableInputs.input_var_names
RiverBedDynamics.input_var_names
(rmg, z) = read_esri_ascii(bedElevation, name='topographic__elevation')
rmg.add_zeros('bed_surface__roughness', at = 'link')
rmg.add_zeros('surface_water__depth', at = 'node')
rmg.add_zeros('rainfall__intensity', at = 'node')
rmg['node']['bed_surface__grain_size_distribution_location'] = np.zeros_like(z)     
rmg['node']['topographic__elevation_original'] = copy.deepcopy(z)               # Converts precipitation from mm/hr to m/s
rmg['node']['bed_surface__grainSizeDistribution_location'] = np.zeros_like(z)     

In [ ]:
## Code Block 4

# Instantiation of components
of = OverlandFlowSpatiallyVariableInputs(rmg, dt_max=max_dt,h_init=0.001)
rbd = RiverBedDynamics(rmg , gsd = gsd, variableCriticalShearStress = True, outletBoundaryCondition='fixedValue')

In [ ]:
## Code Block 6

# Set boundaries as closed boundaries, the outlet is set to an open boundary. 
rmg.set_watershed_boundary_condition(z) 


In [ ]:
## Code Block 7

# Creates the fixed nodes information
fixedNodes = np.zeros_like(z)   # fixedNodes defines as 1 if a node is fixed or 0 if it can vary in elevation
fixedNodes[fixedNodesId] = 1
rmg['node']['bed_surface__fixedElevation'] = fixedNodes                         # Assigns fixed locations to landlab grid 

# Create bed and flow initial condition
rmg['link']['bed_surface__roughness'] = np.zeros(rmg.number_of_links) + n       # n is Manning's roughness coefficient
rmg['node']['rainfall__intensity'][Node_Inlet] = 0.01                           # Precipitation in m/s
rmg['link']['sediment_transport__imposed_sediment_supply'][link_Inlet] = upstreamSedSupply


In [ ]:
## Code Block 8

# Node ID for calculated node elevation
calcNodesId = np.arange(51,57)
nCols = rmg.number_of_node_columns
nRowsCalcNodes = int(calcNodesId.shape[0]/nCols)
calcNodesId = np.reshape(calcNodesId,(nRowsCalcNodes,nCols))

In [ ]:
## Code Block 9

""" Defines some variables to store data """
t = 0                                   # Initializates the variable
storeNow = True
plotNow = True                          # Used to save the plot at time zero
check_tmax = True
tPlotOrg=copy.deepcopy(tPlot)           # A copy of tPlot
storeDataOrg=copy.deepcopy(storeData)   # A copy of tPlot

In [ ]:
## Code Block 10

while t < tmax:
    
    of.overland_flow()  # Runs overland flow for one time step
    rbd.run_one_step()  # Runs riverBedDynamics for one time step
    
    # Gradient preserving at upstream ghost cells
    dsNodesId = np.array(calcNodesId[0,1]-np.arange(1,3)*nCols)
    z = rmg['node']['topographic__elevation']
    bedSlope = (z[dsNodesId[0]] - z[dsNodesId[1]]) / rmg.dx
    
    for i in np.arange(0,calcNodesId.shape[0]):
        rmg['node']['topographic__elevation'][calcNodesId[i,1:nCols-1]] = z[calcNodesId[i,1:nCols-1] - 2*nCols] + 2 * rmg.dx * bedSlope
        
    ## Stores results
    storeData = round(storeData-of.dt, dtPrecision)
    if (storeData <=0) or storeNow:
        print('Storing results at time :',np.round(t,1),' s')
        print('Upstream sediment supply :',upstreamSedSupply,' m2/s')
        data = np.reshape(np.hstack([t,(np.abs(of._q * rmg.dx).T)]),[1,rmg.number_of_links+1])
        with open("output0_links_surface_water__discharge.txt", "ab") as f:
            np.savetxt(f, data,'%.3f')
        data = np.reshape(np.hstack([t,(of._h.T)]),[1,rmg.number_of_nodes+1])
        with open("output1_node_surface_water__depth.txt", "ab") as f:
            np.savetxt(f, data,'%.3f')      
        data = np.reshape(np.hstack([t,np.abs(rbd._tau.T)]),[1,rmg.number_of_links+1])
        with open("output2_link_surface_water__shearStress.txt", "ab") as f:
            np.savetxt(f, data,'%.3f')   
        data = np.reshape(np.hstack([t,rmg.at_node["topographic__elevation"].T]),[1,rmg.number_of_nodes+1])
        with open("output3_node_topographic__elevation.txt", "ab") as f:
            np.savetxt(f, data,'%.3f') 
        data = np.reshape(np.hstack([t,rmg.at_node["bed_surface__medianSize"].T]),[1,rmg.number_of_nodes+1])
        with open("output4_node_bed_surface__medianSize.txt", "ab") as f:
            np.savetxt(f, data,'%.3f')
        data = np.reshape(np.hstack([t,rmg.at_link['sediment_transport__bedloadRate'].T]),[1,rmg.number_of_links+1])
        with open("output5_links_sediment_transport__bedloadRate.txt", "ab") as f:
            np.savetxt(f, data,'%.5f')  
        storeData = round(storeDataOrg, dtPrecision)
        storeNow = False

    tPlot = round(tPlot-of.dt, dtPrecision)
    if tPlot <= 0  or plotNow:
        print('Elapsed time :',np.round(t,1),' s. Current dt =',\
              np.round(of.dt,1),'. Adaptive time =',np.round(of._adaptive_dt,1),' s - Saving plot')
        
        # Water depth plot
        plot_name='Surface water depth [m] at ' + str(np.round(t,0)) + ' sec'
        imshow_grid(rmg, 'surface_water__depth',cmap='Blues',vmin=0,vmax=0.5,plot_name=plot_name)
        output='depth_'+str(np.round(t,0))+'.png'
        plt.savefig(output,dpi=300); plt.close()  
        
        #Bed surface variation plot
        plot_name='Bed surface elevation variation [m] at ' + str(np.round(t,0)) + ' sec'
        ZVar = rmg.at_node["topographic__elevation"] - rmg.at_node['topographic__elevation_original'] 
        imshow_grid(rmg, ZVar,cmap='RdGy',vmin=0,vmax=25,plot_name=plot_name)
        output='topographicVariation_'+str(np.round(t,0))+'.png'
        plt.savefig(output,dpi=300); plt.close()    

        plotNow = False
        tPlot = tPlotOrg

    # Updating t
    if (t + of.dt > tmax) and check_tmax:
        of.dt = tmax - t
        t = tmax
        storeDataNow = True  
        plotNow = True
        check_tmax = False
    else:
        t = round(t + of.dt, dtPrecision)

In [ ]:
###
"""

THIS IS OLD CODE FROM TUTORIAL ONE, I LEFT IT TO CHANGED LATER TO MAKE SOME NEW PLOTS


## Code Block 11

# Results
## Water depth at the end of the simulation
data = np.loadtxt('output1_node_surface_water__depth.txt')
nodesToSample = np.arange(rmg.number_of_node_columns+1,2*rmg.number_of_node_columns)
x = np.arange(0,(rmg.number_of_node_columns-1)*rmg.dx,rmg.dx)

hSample = data[-1,nodesToSample]

hAnalytical = np.zeros_like(x)+h_init
hAnalytical[np.where(x<u*tmax)] =(-7/3 * (n**2 * u**2 * (x[np.where(x<u*tmax)]-u*tmax)))**(3/7)

plt.figure(1)
plt.plot(x, hSample, color='mediumblue')
plt.plot(x, hAnalytical, color='black')
plt.ylabel('Water depth (m)')
plt.xlabel('x (x)')
plt.ylim(0,2.5)
plt.xlim(0,5000)
plt.title('Water depth at 3600 s')
plt.savefig('WaterDepth.png',dpi=300);plt.close()
"""  